# Setting

In [37]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
            .master("spark://10.67.22.4:7077") \
            .appName("Project") \
            .config('spark.jars.packages', 'org.apache.hadoop:hadoop-common:3.2.0')\
            .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')\
            .config('spark.jars.packages', 'com.amazonaws:aws-java-sdk:1.11.375')\
            .config("spark.executor.memory", "2g")\
            .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
            .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
            .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')\
            .config('spark.hadoop.fs.s3a.access.key', '2186eedc366545058b0896dadf79d288')\
            .config('spark.hadoop.fs.s3a.secret.key', 'de147ce2e90741cdb9bae8f53ee5466e')\
            .config('spark.hadoop.fs.s3a.endpoint', 'https://cloud-areapd.pd.infn.it:5210')\
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .config("spark.hadoop.fs.s3a.metadatastore.impl", "org.apache.hadoop.fs.s3a.s3guard.NullMetadataStore") \
            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled","false") \
            .config("com.amazonaws.sdk.disableCertChecking","true") \
            .getOrCreate()

In [38]:
spark

In [39]:
sc = spark.sparkContext
sc

<SparkContext master=spark://10.67.22.4:7077 appName=Project>

In [40]:
raw = spark.read.format("binaryFile").load('s3a://mapd-minidt-batch/data_*.dat')
raw = raw.select("content").rdd.flatMap(lambda el: el)

In [41]:
import struct
def unpack(file_content):
    ppd = []
    word_counter=0
    word_size = 8                                             # size of the word in bytes
    for i in range(0, len(file_content), word_size):
        word_counter+=1
        if word_counter>100: break
        #time.sleep(0.1)
        thisInt = struct.unpack('<q', file_content[i:i+word_size])[0]
        head = (thisInt >> 61) & 0x7
        
        #if head == 2:
        fpga     = (thisInt >> 58) & 0x7
        chan = (thisInt >> 49) & 0x1FF
        orbit  = (thisInt >> 17) & 0xFFFFFFFF
        bx       = (thisInt >> 5 ) & 0xFFF
        tdc = (thisInt >> 0 ) & 0x1F
        ppd.append([head, fpga, chan, orbit, bx, tdc])
    return ppd

In [42]:
data = raw.flatMap(lambda file_content: unpack(file_content)).toDF(('HEAD','FPGA','CHAN','ORBIT','BX','TDC'))

In [43]:
data.show(20)

+----+----+----+---------+----+---+
|HEAD|FPGA|CHAN|    ORBIT|  BX|TDC|
+----+----+----+---------+----+---+
|   1|   0|   0|742070193|   0|  0|
|   0|   0|   0|742070157| 260|  1|
|   4|   0|   0|742070144|1436| 28|
|   5|   0|   0|742070144| 675|  3|
|   2|   1|  11|  5797423| 432|  8|
|   2|   1|  12|  5797423| 431|  9|
|   2|   1|  81|  5797433|1889| 30|
|   2|   1|  98|  5797443| 494| 18|
|   2|   0|  34|  5797432|2379|  2|
|   2|   0|  74|  5797442| 132|  4|
|   2|   0|  75|  5797446|2547| 26|
|   2|   0|  91|  5797454|1509|  2|
|   2|   1|  99|  5797443| 493| 10|
|   2|   1| 100|  5797443| 501| 25|
|   2|   1| 101|  5797443| 503| 14|
|   2|   1|  28|  5797480|2873| 10|
|   2|   1|  14|  5797487| 238| 19|
|   2|   1|  77|  5797506| 142|  5|
|   2|   1| 100|  5797515|3148|  2|
|   2|   1|  51|  5797519|2036|  6|
+----+----+----+---------+----+---+
only showing top 20 rows



In [44]:
data = data.filter(data['HEAD']==2)
data.persist()

DataFrame[HEAD: bigint, FPGA: bigint, CHAN: bigint, ORBIT: bigint, BX: bigint, TDC: bigint]

In [45]:
data.select("CHAN").rdd.max()[0]

22/07/02 09:54:39 ERROR TaskSchedulerImpl: Lost executor 19 on 10.67.22.161: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/07/02 09:54:39 WARN TaskSetManager: Lost task 8.0 in stage 15.0 (TID 294) (10.67.22.161 executor 19): ExecutorLostFailure (executor 19 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/07/02 09:54:39 WARN TaskSetManager: Lost task 11.0 in stage 15.0 (TID 297) (10.67.22.161 executor 19): ExecutorLostFailure (executor 19 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/07/02 09:54:39 WARN TaskSetManager: Lost task 2.0 in stage 15.0 (TID 288) (10.67.22.161 executor 19): ExecutorLostFailure (executor 1

138

DataFrame[HEAD: bigint, FPGA: bigint, CHAN: bigint, ORBIT: bigint, BX: bigint, TDC: bigint]

In [53]:
raw = spark.read.format("binaryFile").option('encoding','uint64').load('s3a://mapd-minidt-batch/data_000000.dat')

In [55]:
raw.show(5)

+--------------------+-------------------+--------+--------------------+
|                path|   modificationTime|  length|             content|
+--------------------+-------------------+--------+--------------------+
|s3a://mapd-minidt...|2022-05-30 08:56:11|10484736|[00 00 82 E7 54 E...|
+--------------------+-------------------+--------+--------------------+



In [54]:
prova = raw.select("content").rdd.flatMap(lambda el: el)#.map(lambda num: '{:064b}'.format(struct.unpack("h", num)[0]))

In [43]:
#prova.take(10)
a = prova.take(20)

In [69]:
type(a[0])

bytearray

In [44]:
a[0][0:4]

bytearray(b'\x00\x00\x82\xe7')

In [45]:
a[0][2]

130

In [39]:
def custom_unpack(num):
    counter = 0
    list_num1, list_num2 = [], []
    while(counter != len(num)):
        num_ex = num[counter:counter+1]
        print('Flag ',counter)
        integer = int(''.join([str(el) for el in num_ex]),16)
        print(integer)
        length = 8
        num_bin = f'{integer:0>8b}'
        list_num1.append(num_bin)
        counter += 1
    counter = 0
    while(counter < len(list_num1)):
        list_num2.append(''.join(list_num1[counter:counter+8]))
        counter += 8
    return list_num2

#custom_unpack([0]*128)
#custom_unpack(['B','C','3','8','6','5','0','2','F','E','B','C','A','5','6','3','3','4','6','3','4','5','7','5','9','7','8','A','E','A','5','0'])
#print(custom_unpack(a[0][0:16]))
#print(a[0][0:16])

In [56]:
prova2 = prova.flatMap(lambda x: custom_unpack(x))

In [66]:
prova2.take(10)

['1110000110000111110011111111100101001101001001001001111001000000',
 '0000100110110110111011011111100101001101001001001110100001000000',
 '0101001001011100111111111111100101001101001001001101110001000000',
 '0101100000011110000000001111101001001101001001000010100001000000',
 '1001110010001011001011011111101001001101001001001110011001000000',
 '1111111001011010001110001111101001001101001001001000111001000000',
 '0101111001001111001111111111101001001101001001001111100001000000',
 '1011010010011110010001101111101001001101001001001110010001000000',
 '0100101010000000111110101111100101001101001001000011011001000100',
 '0000010100110100001000111111101001001101001001001001001001000100']

In [41]:
prova2 = prova2.persist()

In [68]:
prova3 = prova2.map(lambda x: (x[0:5],x[5:17],x[17:49],x[49:58],x[58:61],x[61:]))#.toDF(('TDC','BX','ORBIT','CHAN','FPGA','HEAD'))

In [70]:
prova3.take(2)

[('11100',
  '001100001111',
  '10011111111100101001101001001001',
  '001111001',
  '000',
  '000'),
 ('00001',
  '001101101101',
  '11011011111100101001101001001001',
  '110100001',
  '000',
  '000')]

In [57]:
#prova4 = prova2.map(lambda x: (int(x[0:5],2),int(x[5:17],2),int(x[17:49],2),int(x[49:58],2),int(x[58:61],2),int(x[61:],2))).toDF(('TDC','BX','ORBIT','CHAN','FPGA','HEAD'))
prova4 = prova2.map(lambda x: (int(x[59:],2),int(x[47:59],2),int(x[15:47],2),int(x[6:15],2),int(x[3:6],2),int(x[:3],2))).toDF(('TDC','BX','ORBIT','CHAN','FPGA','HEAD'))

In [59]:
prova4.show(30)

+----+----+----------+----+----+----+
| TDC|  BX|     ORBIT|CHAN|FPGA|HEAD|
+----+----+----------+----+----+----+
| 310|3104|1277368392|   0|   0|   0|
| 260|1552|1295092772| 330|   0|   0|
| 818|1552|2148633636| 112|   2|   5|
| 868|3104|1076041800| 416|   6|   5|
| 356| 490|1617457165| 322|   1|   3|
| 356| 481|2737336333| 328|   4|   4|
|1380| 962|3327189018| 145|   4|   3|
| 868| 485|1787326477| 470|   2|   5|
|4456|1144|1220680707| 162|   6|   5|
| 872| 476| 606629901|   5|   1|   6|
| 872| 960|3524321306| 280|   1|   2|
| 360| 496|1787326477| 278|   4|   4|
| 808|2112|   1666833| 353|   5|   3|
| 864|3104| 537704328| 193|   2|   2|
| 406|2112|2152476433| 139|   0|   0|
| 306|2112|  38547217|   0|   0|   0|
| 768|2112|   4853937| 308|   5|   1|
| 310|2112| 541724849|   0|   0|   0|
| 260| 776|1493778966|  72|   1|   0|
| 818| 776|    606742| 184|   5|   4|
|2404| 484| 713584653| 408|   6|   5|
|1380| 970|1427169306| 176|   1|   0|
| 356| 970|1435557914|  44|   1|   0|
|1380| 966|3

In [58]:
prova4 = prova4.persist()

In [46]:
prova5 = prova4.filter(prova4['HEAD']==2)
prova5 = prova5.persist()

In [61]:
prova5.show(300)
#prova5.count()

+---+---+-----+----+----+----+
|TDC| BX|ORBIT|CHAN|FPGA|HEAD|
+---+---+-----+----+----+----+
+---+---+-----+----+----+----+



In [60]:
prova5 = prova4.filter(prova4['HEAD']==1)
prova5 = prova5.filter(prova5['FPGA']==1)
prova5 = prova5.filter(prova5['TDC']==0)
prova5 = prova5.filter(prova5['CHAN']==100)
prova5 = prova5.filter(prova5['BX']==0)
#prova5 = prova5.filter(prova5['ORBIT']==4083301778)
#prova5 = prova5.filter()
prova5 = prova5.persist()

In [ ]:
raw2 = sc.binaryFiles('s3a://mapd-minidt-batch/data_*.dat')

In [ ]:
raw2.take(3)

In [ ]:
prova_df = prova.toDF("content")
prova_df.printSchema()

In [ ]:
from pyspark.sql import functions as F

prova_df.withColumn("content_s", F.unbase64(F.col("content"))).show(2)

In [ ]:
prova_df.withColumn("content_s", F.decode(F.col("content"),'UTF-8')).show(2)

['1011110000111000011001010000001011111110101111001010010101100011',
 '0011010001100011010001010111010110010111100010101110101001010000']

In [39]:
list_num = ['111','000','AAA']
''.join(list_num[0:4])

'111000AAA'

In [44]:
inte = int('32',16)
type(f'{inte:0>8b}')

str

In [20]:
x=[1,2,3,4,5,6]
x[1:2]

[2]

In [45]:
integer = 0xABC123EFFF
print(type(integer))

<class 'int'>


In [ ]:
raw = sc.binaryFiles('s3a://mapd-minidt-batch/data_*.dat')
raw

In [ ]:
df = spark.read.option("header", False).csv('s3a://mapd-minidt-batch/data_*.dat')

In [ ]:
df = spark.read.format("text").load('s3a://mapd-minidt-batch/data_*.dat')

In [ ]:
df

In [ ]:
df.show(2)

In [ ]:
from pyspark.sql import functions as F

gianni = df.withColumn("_c0s", F.hex(F.col("_c0")))#.show(10)

In [ ]:
from pyspark.sql import functions as f
gianni.select("*", 
          f.substring("_c0s", 0, 1), 
          f.substring("_c0s", 0, 1).cast("int")).show()

In [ ]:
prova = raw.select("content")

In [ ]:
prova.show(1)

In [ ]:
prova_df = prova.toDF("content")
prova_df.printSchema()